# Disclaimer

This notebok created to test my Facial Emotions Recognition model on real images.

I decided to test my model on politics faces and adjusted the categories labels for more fun.

Disclaimer: All  labels on faces are made by machine that was trained on unbiased publically abailable dataset. Author may not agree with the machine conclusions :)

# Face detection

In [ ]:
# face detection function using MTCNN pre-trained model

!pip install mtcnn
!apt-get update
!apt-get install -y libgl1-mesa-glx

import cv2
from mtcnn import MTCNN
from IPython.display import Image

def detect_faces(image_path, detection_confidence=0.99, min_face_size=10):
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Create an MTCNN detector instance
    detector = MTCNN()

    # Use the detector to detect faces in the image
    faces = detector.detect_faces(image)

    # Draw a square around each face
    for face in faces:
        if face['confidence'] < detection_confidence:
            continue
        x, y, width, height = face['box']
        if min(width, height) < min_face_size:
            continue
        face_size = max(width, height)
        x, y = x + (width - face_size) // 2, y + (height - face_size) // 2
        cv2.rectangle(image, (x, y), (x + face_size, y + face_size), (127, 255, 0), 1)

    # Save the image with the detected faces to a file
    cv2.imwrite("detected_faces.jpg", image)

    # Return the path to the saved file
    return "detected_faces.jpg"



In [ ]:
from IPython.display import Image

# Detect faces in the image of various emotions
image_path = detect_faces('/kaggle/input/testimages/pol7.jpg')

# Display the saved image
Image(filename=image_path)

Look, MTCNN model is unable to identify some faces

# Model for Emotions prediction

In [ ]:
# let's see what is in our AffectNet dataset

import pandas as pd
import matplotlib.pyplot as plt  # plot
import os
from os.path import join

path = ('/kaggle/input/affectnet-training-data/')
file = (path + 'labels.csv')
df = pd.read_csv(file)
df.head()

In [ ]:
# display random images

import random

fig, axs = plt.subplots(2, 4, sharey=True, constrained_layout=True, num=None, 
                        figsize=(10, 5), dpi=80, facecolor='gray', edgecolor='k')
fig.suptitle("Sample Faces and Labels")
axs = axs.flatten()


for i in range(8):
    idx = random.randint(0, len(df)-1)  # randomly select an index
    img_path = path + df['pth'][idx]
    img = cv2.imread(img_path)  # read image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # convert to BGR to RGB
    axs[i].imshow(img)
    axs[i].set_title(df['label'][idx])
    axs[i].axis('off')

AffectNet is a large database of faces labeled by "affects" (psychological term for facial expressions). In order to accommodate common memory constraints, the resolution was reduced down to 96x96. Meaning that all images are exactly 96x96 pixels.

# Load images and label categories

In [ ]:
# 1. define functions to pre-process and load images into arrays / label from folders

import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
import os

INPUT_PATH = "/kaggle/input/affectnet-training-data/"
EMOTIONS = [f.name for f in os.scandir(INPUT_PATH) if f.is_dir()]
IMAGE_SIZE = (96, 96)

print(EMOTIONS)

def image_generator(input_path, emotions, image_size):
    for index, emotion in enumerate(emotions):
        for filename in os.listdir(os.path.join(input_path, emotion)):
            img = cv2.imread(os.path.join(input_path, emotion, filename))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
            #img = cv2.resize(img, image_size)
            #img = img.astype('float32') / 255.0  # Normilize
            yield img, index

def load_images(input_path, emotions, image_size):
    X, y = [], []
    for img, label in image_generator(input_path, emotions, image_size):
        X.append(img)
        y.append(label)
    X = np.array(X)
    y = to_categorical(np.array(y))
    return X, y

In [ ]:
# Load the images 
X, y = load_images(INPUT_PATH, EMOTIONS, IMAGE_SIZE)
input_shape = X[0].shape
#input_shape = (96,96,1) 

In [ ]:
# Now let's have some fun and adjust the emotions labels for our politics

EMOTIONS = ['surprise', 'fear', 'neutral', 'sad', 'disgust', 'contempt', 'happy', 'anger']

emotion_map = {
    'surprise': 'Wow!',
    'fear': 'Fear',
    'neutral': "Bored",
    'sad': 'No money',
    'disgust': 'Disgust',
    'contempt': 'Fake',
    'happy': 'Money!',
    'anger': 'Mad'
}

EMOTIONS = [emotion_map[e] for e in EMOTIONS]
EMOTIONS

In [ ]:
# choose a random image index
idx = np.random.randint(len(X))

# display the image and its corresponding label from arrays
plt.imshow(X[idx])
plt.title(EMOTIONS[np.argmax(y[idx])])
plt.axis('off')  # remove the grid
plt.show()

In [ ]:
# Train test split pre-processed data

!pip install scikit-learn

from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Emotions prediction model

In [ ]:
import tensorflow as tf
print("Num TPUs Available: ", len(tf.config.list_logical_devices('TPU')))


In [ ]:
# detect and init the TPU

import tensorflow as tf

tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
# our model_4 taken from Experiments notebook

!pip install keras

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():

    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, Dropout, BatchNormalization, Flatten, Dense, MaxPool2D
    from tensorflow.keras.regularizers import l2
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from tensorflow.keras.optimizers import Adam

    model_4 = Sequential()

    model_4.add(Conv2D(32, (3,3), activation="selu", input_shape=input_shape))
    model_4.add(BatchNormalization())
    model_4.add(MaxPool2D(pool_size=(2,2)))
    model_4.add(Dropout(0.3))

    model_4.add(Conv2D(64, (3,3), activation="selu"))
    model_4.add(BatchNormalization())
    model_4.add(Conv2D(64, (3,3), activation="selu"))
    model_4.add(BatchNormalization())
    model_4.add(MaxPool2D(pool_size=(2,2)))
    model_4.add(Dropout(0.4))

    model_4.add(Conv2D(128, (3,3), activation="selu"))
    model_4.add(BatchNormalization())
    model_4.add(Conv2D(128, (3,3), activation="selu"))
    model_4.add(BatchNormalization())
    model_4.add(MaxPool2D(pool_size=(2,2)))
    model_4.add(Dropout(0.5))

    model_4.add(Conv2D(256, (3,3), activation="selu"))
    model_4.add(BatchNormalization())
    model_4.add(Conv2D(256, (3,3), activation="selu"))
    model_4.add(BatchNormalization())
    model_4.add(MaxPool2D(pool_size=(2,2)))
    model_4.add(Dropout(0.6))

    model_4.add(Flatten())
    model_4.add(Dense(128, activation='selu', kernel_regularizer=l2(0.01)))
    model_4.add(BatchNormalization())
    model_4.add(Dropout(0.5))
    model_4.add(Dense(8, activation='softmax'))

    model_4.compile(optimizer=Adam(learning_rate=0.001), 
                    loss='categorical_crossentropy', 
                    metrics=['accuracy'],
                    steps_per_execution=32)

    model_4.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

BATCH_SIZE = 16 * tpu_strategy.num_replicas_in_sync

history = model_4.fit(X_train, y_train, batch_size=BATCH_SIZE,
                    epochs=30,
                    validation_data=(X_test, y_test),
                    
                    callbacks = [EarlyStopping(patience=10, monitor='val_loss', mode='min'), 
                                 ReduceLROnPlateau(monitor='val_loss', 
                                                   factor=0.5, 
                                                   patience=2, 
                                                   verbose=1),
                                 ModelCheckpoint('best_model.h5', 
                                                 save_best_only=True, 
                                                 save_weights_only=True, 
                                                 monitor='val_accuracy', 
                                                 mode='max')],
                    verbose=1)

In [ ]:
pd.DataFrame(history.history).plot();

In [ ]:
# calculates the false positive rate, true positive rate, and AUC score

from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt

# Make predictions
y_pred = model_4.predict(X_test)

# Compute ROC curve and ROC AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(8):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC AUC score
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel())
roc_auc["micro"] = roc_auc_score(y_test, y_pred, multi_class='ovr')

# Plot the ROC curves for each class and the micro-average ROC curve
plt.figure(figsize=(8, 6))
lw = 2
plt.plot(fpr["micro"], tpr["micro"], lw=lw, label='micro-average ROC curve (AUC = {0:0.2f})'
                                                   ''.format(roc_auc["micro"]))
colors = ['cornflowerblue', 'darkorange', 'forestgreen', 'red', 'purple', 'gray', 'black', 'pink']
for i, color in zip(range(8), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of {0} (AUC = {1:0.2f})'.format(EMOTIONS[i], roc_auc[i]))
    
plt.plot([0, 1], [0, 1], color='gray', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=16)
plt.legend(loc="lower right", fontsize=12)
plt.show()

In [ ]:
# Compute classification report

from sklearn.metrics import classification_report

# Convert one-hot encoded y_test back to integers
y_test_int = np.argmax(y_test, axis=1)

# Make predictions
y_pred = model_4.predict(X_test)

# Convert one-hot encoded y_pred back to integers
y_pred_int = np.argmax(y_pred, axis=1)

# Generate classification report
print(classification_report(y_test_int, y_pred_int))


In [ ]:
# save the model

model_4.save('/kaggle/working/model_4.h5')

# Detect + predict

In [ ]:
# function that use MTCNN face detection and pass detected area to our model_4 for recognition

import cv2
import numpy as np
from mtcnn import MTCNN
from IPython.display import Image

def detect_faces_emo(image_path, detection_confidence=0.99, min_face_size=10):
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Create an MTCNN detector instance
    detector = MTCNN()

    # Use the detector to detect faces in the image
    faces = detector.detect_faces(image)

    # Loop over the detected faces
    for face in faces:
        # Check the confidence score of the detection
        if face['confidence'] < detection_confidence:
            continue
        # Extract the bounding box coordinates
        x, y, width, height = face['box']
        # Check the size of the bounding box
        if min(width, height) < min_face_size:
            continue
        
        # Extract the face region from the image
        face_image = image[y:y+height, x:x+width]
        # Resize the face image to 96x96
        face_image_resized = cv2.resize(face_image, (96, 96))
        #face_image_gray = cv2.cvtColor(face_image_resized, cv2.COLOR_BGR2GRAY)
        # Reshape the face image to match the input shape of the model
        face_image_reshaped = face_image_resized.reshape((1, 96, 96, 3))
        #if np.max(face_image_reshaped) > 1: face_image_reshaped = face_image_reshaped / 255
        
        # Use the model to predict the emotion of the face
        predicted_emo = model_4.predict(face_image_reshaped)[0]
        
        # Draw the predicted emotion label on the rectangle around the face
        label = EMOTIONS[np.argmax(predicted_emo)]
        label_size, baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)
        label_x = x
        label_y = y - 10
        cv2.rectangle(image, (label_x - 5, label_y - label_size[1] - 5), (label_x + label_size[0] + 5, label_y + 5), (204, 229, 255), -1)
        cv2.putText(image, label, (label_x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)



    # Save the image with the detected faces and predicted emotions to a file
    cv2.imwrite("detected_faces.jpg", image)

    # Return the path to the saved file
    return "detected_faces.jpg"



In [ ]:
# Detect faces in the new image of various politics
image_path = detect_faces_emo('/kaggle/input/testimages/pol7.jpg')

# Display the saved image
Image(filename=image_path)

# Adjusted emotions function

In [ ]:
# now let's modify our function to draw the emotion probability out of all 8 emotions

import cv2

def detect_faces_emo_2(image_path, detection_confidence=0.99, min_face_size=10):
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Create an MTCNN detector instance
    detector = MTCNN()

    # Use the detector to detect faces in the image
    faces = detector.detect_faces(image)

    # Loop over the detected faces
    for face in faces:
        # Check the confidence score of the detection
        if face['confidence'] < detection_confidence:
            continue
        # Extract the bounding box coordinates
        x, y, width, height = face['box']
        # Check the size of the bounding box
        if min(width, height) < min_face_size:
            continue
        
        # Extract the face region from the image
        face_image = image[y:y+height, x:x+width]
        # Resize the face image to 96x96
        face_image_resized = cv2.resize(face_image, (96, 96))
        # Reshape the face image to match the input shape of the model
        face_image_reshaped = face_image_resized.reshape((1, 96, 96, 3))
        # Use the model to predict the emotion of the face
        predicted_emo = model_4.predict(face_image_reshaped)[0]
        predicted_emo_sorted = sorted(list(enumerate(predicted_emo)), key=lambda x: x[1], reverse=True)
        
        # Extract the predicted probabilities for each emotion category
        # Extract the predicted probabilities for each emotion category
        probabilities = ["{}".format(round(prob * 100)) for index, prob in predicted_emo_sorted]
        
        # Draw the predicted emotion label on the rectangle around the face
        label = EMOTIONS[np.argmax(predicted_emo)]
        label_size, baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)
        label_x = x
        label_y = y - 10
        cv2.rectangle(image, (label_x - 5, label_y - label_size[1] - 5), (label_x + label_size[0] + 5, label_y + 5), (204, 229, 255), -1)
        cv2.putText(image, label, (label_x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
        
        # Draw a square rectangle around the face
        face_size = min(width, height)
        x_center = x + int(width / 2)
        y_center = y + int(height / 2)
        x1 = x_center - int(face_size / 2)
        y1 = y_center - int(face_size / 2)
        x2 = x_center + int(face_size / 2)
        y2 = y_center + int(face_size / 2)
        cv2.rectangle(image, (x1, y1), (x2, y2), (127, 255, 0), 2)

        # Draw a vertical table with the predicted emotion probabilities
        table_x, table_y = x1, y2 + 10
        for index, prob in predicted_emo_sorted:
            table_y += 10
            emotion = EMOTIONS[index]
            cv2.putText(image, emotion, (table_x, table_y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1)
            cv2.putText(image, "{}%".format(round(prob * 100)), (table_x + 50, table_y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1)
    
    # Save the image with the detected faces and predicted emotions to a file
    cv2.imwrite("detected_faces.jpg", image)

    # Return the path to the saved file
    return "detected_faces.jpg"


In [ ]:
# Detect faces in the image of politics
image_path = detect_faces_emo_2('/kaggle/input/testimages/pol7.jpg')

# Display the saved image
Image(filename=image_path)

In [ ]:
# Detect faces in the image of politics
image_path = detect_faces_emo_2('/kaggle/input/testimages/pol6.jpg')

# Display the saved image
Image(filename=image_path)

In [ ]:
# Detect faces in the image of politics
image_path = detect_faces_emo_2('/kaggle/input/testimages/pol10.jpg')

# Display the saved image
Image(filename=image_path)

In [ ]:
# Detect faces in the image of politics
image_path = detect_faces_emo_2('/kaggle/input/testimages/pol12.jpg')

# Display the saved image
Image(filename=image_path)

What if we train the model on politicians only? 

What if we process the video data and add time dimension to the model? (LSTM)

What if we design the feature called "Lie detection" based on facial expressions in certain situations?

What are your ideas? Please comment and vote!